In [1]:
# !pip install Flask==0.12 requests request line-bot-sdk
# !pip install keras==2.4.2
# !pip install pillow
# !pip install tensorflow==2.2.0
# !pip install matplotlib

In [2]:
from flask import Flask, request, abort
import requests
import tensorflow as tf
import numpy as np
import time
import os
import get_FoodData_from_mySQL
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
    ImageMessage)
import foodrecord

In [3]:
'''

整體功能描述

'''

'\n\n整體功能描述\n\n'

In [4]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r',encoding='utf8'))
server_url=secretFileContentJson.get("server_url")

#複製的，載入基礎設定檔
channel_access_token = secretFileContentJson.get("channel_access_token")
self_user_id = secretFileContentJson.get("self_user_id")
rich_menu_id = secretFileContentJson.get("rich_menu_id")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/素材" , static_folder = "./素材/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [5]:
# 圖片預處理,將圖片整理成模型可以接受的格式
def read_image(img_path):
    try:
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(299, 299))
    except Exception as e:
        print(img_path,e)

    img = tf.keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return img/255

In [6]:
#處理影像訊息
@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
#     #不處理官方發來的訊息
#     if event.source.user_id != "Udeadbeefdeadbeefdeadbeefdeadbeef":
    #使用影像id拿到影像內容
    message_content = line_bot_api.get_message_content(event.message.id) #event.message.id得到傳來的影像id

    #將影像內容存成jpg檔
    tempfile_path = os.path.join("./tmp_for_linebot", event.message.id+".jpg")
    with open(tempfile_path, 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

#     line_bot_api.reply_message(
#         event.reply_token,
#         TextSendMessage(text="辨識中....")
#     )
    
    start = time.process_time()

    #貼上訓練時的分類字典

    labels = {'香蕉': 0, '炒麵': 1, '沙拉': 2, '咖哩飯': 3, '雞翅': 4, '三明治': 5, '炒飯': 6, '餃子': 7, '拉麵': 8}
    labels = {str(v): k for k, v in labels.items()}

    #載入模型
    model = tf.keras.models.load_model('model_trained_9class20200622.h5')  # 辨識食譜
    #model = tf.keras.models.load_model('myiv3LeafFinetune_employee.h5')
#     line_bot_api.reply_message(
#         event.reply_token,
#         TextSendMessage(text="模型載入完成")
#     )

    
    img = read_image(tempfile_path)

    #推論
    pred = model.predict(img)[0]

    # 推論出機率最高的分類, 取得所在位置
    index = np.argmax(pred)
    if pred[index] > 0.8:
        print('圖片與  ', labels[str(index)],'  相似度為:   ', round(pred[index]*100, 2),"%")
        end = time.process_time()
        #產生推論報告文字
        response_text = "這張圖有{0}%是{1}，分析時間為{2}秒{3}".format(
            round(pred[index] * 100, 1),labels[str(index)], round(end - start), chr(0x1F436))
        
    else:
        print('圖片無法辨識', '與', labels[str(index)],'相似度為:', round(pred[index]*100, 2),"%")
        end = time.process_time()
        #產生推論報告文字
        response_text = "辨識度不足!這張圖只有{0}%是{1}，分析時間為{2}秒{3}".format(
            round(pred[index] * 100, 1),labels[str(index)], round(end - start), chr(0x1F436))

    print('推論圖片花費時間:', round(end - start), '秒')

#     #產生推論報告文字
#     response_text = "這張圖有{0}%是{1}，分析時間為{2}秒{3}".format(
#         round(pred[index] * 100, 1),labels[str(index)], round(end - start), chr(0x1F436))

        #將報告文字推送到Line聊天室
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=response_text)
    )

In [7]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage,VideoSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    with open(fileName,encoding='utf-8') as f:
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))      
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'video':
            returnArray.append(VideoSendMessage.new_from_json_dict(jsonObject))    


    # 回傳
    return returnArray

In [8]:
'''

handler處理關注消息

用戶關注時，讀取 素材 -> 關注 -> reply.json

將其轉換成可寄發的消息，傳回給Line

'''

# 引用套件
from linebot.models import (
    FollowEvent
)

# 關注事件處理
@handler.add(FollowEvent)
def process_follow_event(event):
    
    # 讀取並轉換
    result_message_array =[]
    replyJsonPath = "素材/關注/reply.json"
    result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
    # 將用戶資訊存在檔案內
    with open("./users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
    
    #複製的 將菜單綁至用戶上
    user_id = line_bot_api.get_profile(event.source.user_id)
    linkRichMenuId = rich_menu_id
    linkMenuEndpoint = 'https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
    linkMenuRequestHeader = {'Content-Type': 'image/jpeg', 'Authorization': 'Bearer %s' % channel_access_token}
    lineLinkMenuResponse = requests.post(linkMenuEndpoint, headers=linkMenuRequestHeader)
    app.logger.info("Link Menu to %s status :" % user_id, lineLinkMenuResponse)

    # 消息發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )


In [9]:
'''

handler處理文字消息

收到用戶回應的文字消息，
按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的reply.json

轉譯json後，將消息回傳給用戶

'''

# 引用套件
from linebot.models import (
    MessageEvent, TextMessage
)

# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):
    msg = str(event.message.text).upper().strip()
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
    profile_dict = vars(user_profile)         
    ID = profile_dict.get("user_id")#

        
    # 讀取本地檔案，並轉譯成消息
    result_message_array =[]
#         replyJsonPath = "素材/顯示食譜推薦/reply.json"
    replyJsonPath = "素材/"+event.message.text+"/reply.json"

    #如果文字消息為"顯示食譜推薦"，啟動隨機輪播功能     
    if replyJsonPath == "素材/顯示食譜推薦/reply.json":
        build_menu = get_FoodData_from_mySQL.get_food_data()
        build_menu
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

        # 發送
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array)

    elif msg == "顯示飲食運動紀錄":
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="顯示飲食運動紀錄aaaaa"))
        btn_msg = foodrecord.stock_reply_other_news(msg)
        line_bot_api.push_message(ID, btn_msg)



    else:
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        # 發送
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array)

    


In [11]:
'''

handler處理Postback Event

載入功能選單與啟動特殊功能

解析postback的data，並按照data欄位判斷處理

現有三個欄位
menu, folder, tag

若folder欄位有值，則
    讀取其reply.json，轉譯成消息，並發送

若menu欄位有值，則
    讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
    讀取其reply.json，轉譯成消息，並發送

'''
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs 

@handler.add(PostbackEvent)
def process_postback_event(event):
    


    query_string_dict = parse_qs(event.postback.data)
    
    print(query_string_dict)
    if 'folder' in query_string_dict:
    
        result_message_array =[]

        replyJsonPath = '素材/'+query_string_dict.get('folder')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    elif 'menu' in query_string_dict:
 
        linkRichMenuId = open("素材/"+query_string_dict.get('menu')[0]+'/rich_menu_id', 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
        replyJsonPath = '素材/'+query_string_dict.get('menu')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )

In [12]:
'''

Application 運行（開發版）

'''
if __name__ == "__main__":
    app.run(port=5000, host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.18.0.1 - - [14/Jul/2020 03:08:06] "POST / HTTP/1.1" 200 -


In [13]:
'''

Application 運行（heroku版）

'''

# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])

'\n\nApplication 運行（heroku版）\n\n'